In [1]:
import time
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import math
import tensorflow as tf
import tensorflow.contrib.seq2seq as seq2seq
import pprint
pp = pprint.PrettyPrinter(indent=4)

from tensorflow.contrib.rnn import MultiRNNCell, RNNCell, DropoutWrapper, LSTMCell, GRUCell, LSTMStateTuple, MultiRNNCell, DropoutWrapper, LayerNormBasicLSTMCell
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

C:\Users\kbs\Anaconda3_64\lib\site-packages\sklearn\utils\fixes.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [3]:
start_time = time.time()

In [4]:
def create_dataset(dataset, look_back=1):
    x, y = [], []
    until = len(dataset)-look_back-1
    for i in range(until):
        x.append(dataset[i:(i+look_back), 0])
        y.append(dataset[i + look_back, 0])
    return np.array(x), np.array(y)

# fix random seed for reproducibility

In [5]:
np.random.seed(7)

# load the dataset

In [1]:
df = pd.read_csv('kaggle_bitcoin_from_2017_04_01.csv', usecols=[7], engine='python')
len(df)

NameError: name 'pd' is not defined

In [7]:
df = df.dropna(thresh=1) # 그냥 NaN value를 drop하면 안될 것 같다. 평균 등을 이용해서 전처리 단계에서 NaN 값을 더 채울 수 있도록 보강하자.
len(df)

NameError: name 'df' is not defined

In [8]:
dataset = df.values
dataset = dataset.astype('float32')

# normalize the dataset

In [9]:
#TODO: Y는 scale 제외
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split into train and test sets

In [10]:
train_size = int(len(dataset) * 0.8) # train, validation, test 3개 셋으로 나눠서 오버피팅, 언더피팅 상태를 좀 확인해보면 좋을 것 같다.
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [11]:
train[:10]

array([[ 0.00511307],
       [ 0.00517356],
       [ 0.00534809],
       [ 0.00551021],
       [ 0.00633264],
       [ 0.0055353 ],
       [ 0.00631988],
       [ 0.00575346],
       [ 0.00524336],
       [ 0.00558454]], dtype=float32)

In [12]:
test[:2]

array([[ 0.54128867],
       [ 0.5403586 ]], dtype=float32)

# reshape into X=t and Y=t+1

In [13]:
look_back = 5
train_x, train_y = create_dataset(train, look_back)
test_x, test_y = create_dataset(test, look_back)

In [14]:
train_x

array([[ 0.00511307,  0.00517356,  0.00534809,  0.00551021,  0.00633264],
       [ 0.00517356,  0.00534809,  0.00551021,  0.00633264,  0.0055353 ],
       [ 0.00534809,  0.00551021,  0.00633264,  0.0055353 ,  0.00631988],
       ..., 
       [ 0.54168195,  0.54295045,  0.54126304,  0.54312426,  0.54375046],
       [ 0.54295045,  0.54126304,  0.54312426,  0.54375046,  0.54427153],
       [ 0.54126304,  0.54312426,  0.54375046,  0.54427153,  0.5441249 ]], dtype=float32)

# parameters

In [15]:
## Model structure parameters
in_size = look_back # Size of input vectors at each time step
hidden_size = 64 # Size of hidden state vector
num_layers = 1 # Number of hidden layers
out_size = 1 # Size of output vectors at each time step

learning_rate = 0.001 # Learning rate

# Data and train parameters
batch_size = 64 # Training batch size
time_steps = 50 # (Maximum) number of time steps in each batch
num_epochs = 1000
display_interval = 20

In [16]:
# Placeholder for inputs: shape [batch_size, timesteps, in_size]
X = tf.placeholder(tf.float32, shape=[None, None, in_size], name='input_X')

# Placeholder for outputs: shape [batch_size, timesteps, in_size]
Y = tf.placeholder(tf.float32, shape=[None, None, out_size], name='target_Y')

# Placeholder for initial state
state_size = num_layers * 2 * hidden_size
hidden_init = tf.placeholder(tf.float32, shape=[None, state_size])

# create and fit the LSTM network with tensorflow

In [17]:
hidden_cells = [LSTMCell(num_units=hidden_size, state_is_tuple=True) for i in range(num_layers)]
hidden = MultiRNNCell(hidden_cells, state_is_tuple=True)

outputs, _states = tf.nn.dynamic_rnn(cell=hidden,
                                     dtype=tf.float32,
                                     inputs=X)

W = tf.get_variable(name='weights', 
                    shape=[hidden_size, out_size], 
                    initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable(name='biases', 
                    shape=[out_size], 
                    initializer=tf.constant_initializer(0.0))

outputs_reshaped = tf.reshape(outputs, [-1, hidden_size])
logits = tf.nn.xw_plus_b(x=outputs_reshaped, weights=W, biases=b, name='logits')

batch_time_shape = tf.shape(outputs)
outputs_activated = tf.reshape(tensor=tf.nn.softmax(logits), 
                               shape=[batch_time_shape[0], batch_time_shape[1], out_size])

In [18]:
print(X)
print(Y)

print(outputs)
print(outputs_reshaped)

print(logits)

Tensor("input_X:0", shape=(?, ?, 5), dtype=float32)
Tensor("target_Y:0", shape=(?, ?, 1), dtype=float32)
Tensor("rnn/transpose:0", shape=(?, ?, 64), dtype=float32)
Tensor("Reshape:0", shape=(?, 64), dtype=float32)
Tensor("logits:0", shape=(?, 1), dtype=float32)


In [19]:
Y_batch_flatten = tf.reshape(Y, [-1, out_size])
cost = tf.reduce_mean(tf.square(logits - Y_batch_flatten))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# make predictions

In [20]:
batch_x = np.zeros([batch_size, time_steps, in_size])
batch_y = np.zeros([batch_size, time_steps, out_size])

In [21]:
print(batch_x.shape, batch_y.shape)

(64, 50, 5) (64, 50, 1)


In [22]:
train_y

array([ 0.0055353 ,  0.00631988,  0.00575346, ...,  0.54427153,
        0.5441249 ,  0.54252273], dtype=float32)

In [23]:
possible_batch_idx = range(train_x.shape[0] - time_steps - 1)

# Declare the session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(num_epochs):
        batch_id = random.sample(population=possible_batch_idx, 
                                 k=batch_size)

        for j in range(time_steps):
            idx_X = [k + j for k in batch_id]
            idx_Y = [k + j + 1 for k in batch_id]

            batch_x[:, j, :] = dataset[idx_X, :]
            batch_y[:, j, :] = dataset[idx_Y, :]

        init_value = np.zeros((batch_x.shape[0], state_size))
        training_cost, _ = sess.run([cost, train_op], feed_dict={X:batch_x, Y:batch_y, hidden_init:init_value})

        if i % display_interval == 0:
            print("epoch: ", i, "\tcost: ", training_cost)
          
        # TODO:model saver
#         if i % display_interval == 0:
#             print("epoch: ", i, "\tcost: ", training_cost)

epoch:  0 	cost:  0.0256208
epoch:  20 	cost:  0.00139457
epoch:  40 	cost:  0.00130842
epoch:  60 	cost:  0.000915034
epoch:  80 	cost:  0.000829206
epoch:  100 	cost:  0.000760902
epoch:  120 	cost:  0.000572091
epoch:  140 	cost:  0.000506475
epoch:  160 	cost:  0.00038207
epoch:  180 	cost:  0.000309584
epoch:  200 	cost:  0.000436431
epoch:  220 	cost:  0.000270539
epoch:  240 	cost:  0.000259
epoch:  260 	cost:  0.000269103
epoch:  280 	cost:  0.000216526
epoch:  300 	cost:  0.000153397
epoch:  320 	cost:  0.000208664
epoch:  340 	cost:  0.000135264
epoch:  360 	cost:  0.000133325
epoch:  380 	cost:  0.000107058
epoch:  400 	cost:  7.84301e-05
epoch:  420 	cost:  7.29715e-05
epoch:  440 	cost:  7.2327e-05
epoch:  460 	cost:  5.3754e-05
epoch:  480 	cost:  4.35301e-05
epoch:  500 	cost:  4.36992e-05
epoch:  520 	cost:  3.78462e-05
epoch:  540 	cost:  2.8145e-05
epoch:  560 	cost:  2.43939e-05
epoch:  580 	cost:  2.27302e-05
epoch:  600 	cost:  2.13627e-05
epoch:  620 	cost:  1.860